In [1]:
import os
import xarray as xr

from sklearn.metrics import mean_squared_error

# pytorch
import torch
from torch import nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Eurac utils
from hython.hython.datasets.dataset_reader import xarray_to_array, create_dataset
from hython.hython.models.lstm import CustomLSTMModel
from hython.hython.train_val import train_val

import warnings
warnings.filterwarnings("ignore")

In [2]:
import glob 
glob.glob('/mnt/CEPH_PROJECTS/InterTwin/Surrogate_Model/Train Model/Deltras Data')

['/mnt/CEPH_PROJECTS/InterTwin/Surrogate_Model/Train Model/Deltras Data']

In [3]:
dyn_vars_ds = xr.open_dataset('/mnt/CEPH_PROJECTS/InterTwin/Surrogate_Model/Train Model/Deltras Data/Dynamic_Data_2000_2015.nc',decode_coords='all')#.to_dataset(dim='variable')
static_params_ds = xr.open_dataset('/mnt/CEPH_PROJECTS/InterTwin/Surrogate_Model/Train Model/Deltras Data/staticmaps.nc',decode_coords='all')#.to_dataset(dim='variable')
target_ds = xr.open_dataset('/mnt/CEPH_PROJECTS/InterTwin/Surrogate_Model/Train Model/Deltras Data/Target_train_et_sm_2000_2015.nc',decode_coords='all')

In [4]:
stat_parmas = ['dem_subgrid','KsatVer_100.0cm', 'wflow_dem','hydrodem_avg_D8', 'N',
               'RootingDepth', 'thetaS','thetaR','KsatVer', 'M_original_', 'f_', 'M_original', 'f']

dyn_vars_ds1 = dyn_vars_ds.sel(time=slice(dyn_vars_ds.time[0], dyn_vars_ds.time[1459]))
target_ds1 = target_ds.sel(time=slice(target_ds.time[0], target_ds.time[1459]))


In [5]:
dyn_vars_ds1

<xarray.Dataset>
Dimensions:      (time: 1460, lat: 214, lon: 245)
Coordinates:
  * time         (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2003-12-30
  * lat          (lat) float64 47.09 47.08 47.07 47.06 ... 45.33 45.32 45.31
  * lon          (lon) float64 10.29 10.3 10.3 10.31 ... 12.3 12.3 12.31 12.32
Data variables:
    precip       (time, lat, lon) float32 ...
    pet          (time, lat, lon) float32 ...
    temp         (time, lat, lon) float32 ...
    spatial_ref  (time) int32 ...
Attributes:
    units:           mm d**-1
    long_name:       Total precipitation
    category:        meteo
    notes:           Extracted from Copernicus Climate Data Store; resampled ...
    paper_doi:       10.1002/qj.3803
    paper_ref:       Hersbach et al. (2019)
    source_license:  https://cds.climate.copernicus.eu/cdsapp/#!/terms/licenc...
    source_url:      https://doi.org/10.24381/cds.bd0915c6
    source_version:  ERA5 daily data on pressure levels
    _FillValue:      nan
    unit:            mm
    precip_fn:       era5

In [59]:
# # Create the dataset and return DataLoader
target_arr, static_params_arr, dyn_vars_arr   = create_dataset(dyn_vars_ds1, static_params_ds, target_ds1,
                                            batch_size=8,
                                            time_steps=1460, #time_steps=365 5844
                                            dyn_vars_names=['precip', 'pet', 'temp'],
                                            static_params_names=stat_parmas, #[ 'M', 'thetaS', 'RootingDepth', 'Kext', 'Sl', 'Swood', 'TT', 'KsatHorFrac'],
                                            target_names=['soil_moisture', 'evapotranspiration']) # ['vwc_percroot', 'soil_moisture', 'evapotranspiration']


import numpy as np
# #/mnt/CEPH_PROJECTS/InterTwin/Surrogate_Model/Train Model/preprocessed_data/target_arr.npy
np.save('/mnt/CEPH_PROJECTS/InterTwin/Surrogate_Model/Train Model/preprocessed_data/static_params_arr_4years.npy', static_params_arr)
np.save('/mnt/CEPH_PROJECTS/InterTwin/Surrogate_Model/Train Model/preprocessed_data/target_arr_4years.npy', target_arr)
np.save('/mnt/CEPH_PROJECTS/InterTwin/Surrogate_Model/Train Model/preprocessed_data/dyn_vars_arr_4years.npy', dyn_vars_arr)

(52430, 1460, 2)
Done target parmas
(52430, 1460, 3)
Done dynamics parmas
(52430, 13)
Done static parmas
0
0
0


In [60]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [61]:
# x1 = dyn_vars_arr
# x2 = static_params_arr

In [62]:
static_params_arr1 = static_params_arr.unsqueeze(1).expand(-1, dyn_vars_arr.shape[1], -1)
all_vars_arr = torch.cat((dyn_vars_arr, static_params_arr1), -1)
all_vars_arr.shape

torch.Size([20051, 1460, 16])

In [70]:
validation_seq_length = 365
batch_size = 8

# Split the data into training and validation sets
train_data = dyn_vars_arr[:, :-validation_seq_length, :]
train_targets = target_arr[:, :-validation_seq_length, :]

val_data = dyn_vars_arr[:, -validation_seq_length:, :]
val_targets = target_arr[:, -validation_seq_length:, :]


# Create DataLoader for batching
train_dataset = TensorDataset(train_data, train_targets)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(val_data, val_targets)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)


In [71]:
train_data.shape

torch.Size([20051, 1095, 3])

In [72]:
input_size = 3
hidden_size = 64
num_layers = 2
output_size = 2


In [73]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out

# Instantiate the model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [74]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        
        self.mseloss = nn.MSELoss()

    def forward(self, y_true, y_pred):
        """
        Calculate the Root Mean Squared Error (RMSE) between two tensors.

        Parameters:
        y_true (torch.Tensor): The true values.
        y_pred (torch.Tensor): The predicted values.

        Returns:
        torch.Tensor: The RMSE loss.
        """
        rmse_loss = torch.sqrt(self.mseloss(y_true, y_pred))

        return rmse_loss
    


In [79]:
# Loss and optimizer
learning_rate = 0.01
criterion = RMSELoss() #nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [80]:

############### just the dynamic variables ####################


# Training loop with RMSE monitoring
train_losses = []  # To store training losses
val_losses = []    # To store validation losses
val_rmse_values = []  # To store RMSE values for the first target during validation

for epoch in range(num_epochs):
    print(f'######### Epoch [{epoch+1}/{num_epochs}]')
    model.train()
    train_loss_sum = 0.0
    num_batches = 0

    for inputs, targets in train_dataloader:
        inputs = inputs.view(-1, seq_length - validation_seq_length, input_size)
        targets = targets.view(-1, seq_length - validation_seq_length, output_size)
        outputs = model(inputs.to(device))
        loss = criterion(outputs, targets.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss_sum += loss.item()
        num_batches += 1

    train_loss_avg = train_loss_sum / num_batches
    train_losses.append(train_loss_avg)

    #if (epoch + 1) % 10 == 0:
    print(f'Train Loss: {train_loss_avg:.4f}')

#     # Validation
    model.eval()
    vals = []
    with torch.no_grad():
        for inputs, _ in val_dataloader:
            inputs = inputs.view(-1, validation_seq_length, input_size)
            val_outputs = model(inputs.to(device)).cpu().numpy()
            vals.append(val_outputs)
            
        val_predictions = np.vstack(vals)
        
        rmse_metric = mean_squared_error(val_targets[:,:,0], val_predictions[:,:,0], squared=False)
        val_rmse_values.append(rmse_metric)
        print(f'Validation RMSE: {rmse_metric}')


######### Epoch [1/50]
Train Loss: 7.7360
Validation RMSE: 29.330810546875
######### Epoch [2/50]
Train Loss: 7.6811
Validation RMSE: 29.457853317260742
######### Epoch [3/50]
Train Loss: 7.5923
Validation RMSE: 27.814800262451172
######### Epoch [4/50]
Train Loss: 7.5806
Validation RMSE: 30.020627975463867
######### Epoch [5/50]
Train Loss: 7.5392
Validation RMSE: 29.01734733581543
######### Epoch [6/50]
Train Loss: 7.5676
Validation RMSE: 26.378807067871094
######### Epoch [7/50]
Train Loss: 7.4542
Validation RMSE: 24.62902069091797
######### Epoch [8/50]
Train Loss: 7.4299
Validation RMSE: 26.0306339263916
######### Epoch [9/50]
Train Loss: 7.4829
Validation RMSE: 26.76091957092285
######### Epoch [10/50]
Train Loss: 7.4695
Validation RMSE: 23.515878677368164
######### Epoch [11/50]
Train Loss: 7.5039
Validation RMSE: 24.625141143798828
######### Epoch [12/50]
Train Loss: 7.4686
Validation RMSE: 26.890583038330078
######### Epoch [13/50]
Train Loss: 7.5032
Validation RMSE: 24.290369

In [69]:
############### with the static variables ####################

# Training loop with RMSE monitoring
train_losses = []  # To store training losses
val_losses = []    # To store validation losses
val_rmse_values = []  # To store RMSE values for the first target during validation

for epoch in range(num_epochs):
    print(f'######### Epoch [{epoch+1}/{num_epochs}]')
    model.train()
    train_loss_sum = 0.0
    num_batches = 0

    for inputs, targets in train_dataloader:
        inputs = inputs.view(-1, seq_length - validation_seq_length, input_size)
        targets = targets.view(-1, seq_length - validation_seq_length, output_size)
        outputs = model(inputs.to(device))
        loss = criterion(outputs, targets.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss_sum += loss.item()
        num_batches += 1

    train_loss_avg = train_loss_sum / num_batches
    train_losses.append(train_loss_avg)

    #if (epoch + 1) % 10 == 0:
    print(f'Train Loss: {train_loss_avg:.4f}')

#     # Validation
    model.eval()
    vals = []
    with torch.no_grad():
        for inputs, _ in val_dataloader:
            inputs = inputs.view(-1, validation_seq_length, input_size)
            val_outputs = model(inputs.to(device)).cpu().numpy()
            vals.append(val_outputs)
            
        val_predictions = np.vstack(vals)
        
        rmse_metric = mean_squared_error(val_targets[:,:,0], val_predictions[:,:,0], squared=False)
        val_rmse_values.append(rmse_metric)
        print(f'Validation RMSE: {rmse_metric}')


######### Epoch [1/50]
Train Loss: 15.6415
Validation RMSE: 20.020015716552734
######### Epoch [2/50]
Train Loss: 15.3132
Validation RMSE: 19.903535842895508
######### Epoch [3/50]
Train Loss: 15.3060
Validation RMSE: 19.682777404785156
######### Epoch [4/50]
Train Loss: 15.3088
Validation RMSE: 19.937238693237305
######### Epoch [5/50]
Train Loss: 15.3103
Validation RMSE: 20.335386276245117
######### Epoch [6/50]
Train Loss: 15.3074
Validation RMSE: 19.881832122802734
######### Epoch [7/50]
Train Loss: 15.3056
Validation RMSE: 19.88032341003418
######### Epoch [8/50]
Train Loss: 15.3015
Validation RMSE: 20.159563064575195
######### Epoch [9/50]
Train Loss: 15.3011
Validation RMSE: 19.980215072631836
######### Epoch [10/50]
Train Loss: 15.2840
Validation RMSE: 19.718652725219727
######### Epoch [11/50]
Train Loss: 15.2925
Validation RMSE: 20.010536193847656
######### Epoch [12/50]
Train Loss: 15.2916


KeyboardInterrupt: 

In [20]:
# # Training loop
# num_epochs = 50
# seq_length = 1460

# for epoch in range(num_epochs):
#     for inputs, targets in train_dataloader:
#         inputs = inputs.view(-1, seq_length - validation_seq_length, input_size)
#         targets = targets.view(-1, seq_length - validation_seq_length, output_size)
#         outputs = model(inputs.to(device))
#         loss = criterion(outputs, targets.to(device))
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     #if (epoch + 1) % 10 == 0:
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/50], Loss: 17.3820
Epoch [2/50], Loss: 16.8013
Epoch [3/50], Loss: 4.9605
Epoch [4/50], Loss: 18.3184
Epoch [5/50], Loss: 12.2493
Epoch [6/50], Loss: 17.9480
Epoch [7/50], Loss: 9.5517
Epoch [8/50], Loss: 19.2007
Epoch [9/50], Loss: 6.9690
Epoch [10/50], Loss: 5.7845
Epoch [11/50], Loss: 17.7766


KeyboardInterrupt: 

### Old

In [83]:
# Create the dataset and return DataLoader
train_loader, val_loader  = create_dataset(dyn_vars_ds1, static_params_ds, target_ds1,
                                           batch_size=8,
                                           time_steps=1460, #time_steps=365 5844
                                           dyn_vars_names=['precip', 'pet', 'temp'],
                                           static_params_names=stat_parmas, #[ 'M', 'thetaS', 'RootingDepth', 'Kext', 'Sl', 'Swood', 'TT', 'KsatHorFrac'],
                                           target_names=['soil_moisture', 'evapotranspiration']) # ['vwc_percroot', 'soil_moisture']

(52430, 1095, 2)
Done target parmas
(52430, 1095, 3)
Done dynamics parmas
(52430, 13)
Done static parmas
0
0
0


In [46]:
# dyn_vars_ds = xr.open_dataset('./data/dyn_vars.nc',decode_coords='all')#.to_dataset(dim='variable')
# static_params_ds = xr.open_dataset('./data/staticmaps_calibrated_parameters.nc',decode_coords='all')#.to_dataset(dim='variable')
# target_ds = xr.open_dataset('./data/soil_moisture_2019.nc',decode_coords='all')

# # Create the dataset and return DataLoader
# train_loader, val_loader  = create_dataset(dyn_vars_ds, static_params_ds, target_ds,batch_size=8,
#                                            time_steps=365, #time_steps=365 5844
#                                            dyn_vars_names=['precip', 'pet', 'temp'],
#                                            static_params_names=[ 'M', 'thetaS', 'RootingDepth', 'Kext', 'Sl', 'Swood', 'TT', 'KsatHorFrac'],
#                                            target_names=['vwc_percroot']) # ['vwc_percroot']

In [84]:
# Invistigate the dataset 
for x1, x2, y in train_loader:
    print(x1.shape)
    print(x2.shape)
    print(y.shape)
    break

torch.Size([8, 1095, 3])
torch.Size([8, 13])
torch.Size([8, 1095, 2])


In [85]:
# # Create the model 
# model_params={
#     "input_size": 3, #number of dynamic predictors - user_input 
#     "number_static_predictors": 85, #number of static parameters - user_input 
#     "hidden_size": 256, # user_input
#     "output_size": 2, # number_target - user_input

# }

# model = CustomLSTMModel(model_params)
# model

In [86]:
# from hython.hython.models.CudnnLstmModel import CudnnLstmModel
# nx = 3 + 2
# ny = 1
# hiddenSizeLst = 256
# WARM_UP_DAY = 10

# model = CudnnLstmModel(nx=nx, ny=ny, hiddenSize=hiddenSizeLst)
# model

In [87]:
def mse_metric(output, target):
    metric_epoch = mean_squared_error(output[:,:,0], target[:,:,0], squared=False)
    return metric_epoch


class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        
        self.mseloss = nn.MSELoss()

    def forward(self, y_true, y_pred):
        """
        Calculate the Root Mean Squared Error (RMSE) between two tensors.

        Parameters:
        y_true (torch.Tensor): The true values.
        y_pred (torch.Tensor): The predicted values.

        Returns:
        torch.Tensor: The RMSE loss.
        """
        rmse_loss = torch.sqrt(self.mseloss(y_true, y_pred))

        return rmse_loss

In [88]:
# # Training and validation 
# path2models= "./checkpoints" #./output/kaggle/working/AI4EO/models
# if not os.path.exists(path2models):
#     os.mkdir(path2models)
    
    
# ## Where to save the trained models weights 
# ## Set the optimization algorithms and learning rate
# opt = optim.Adam(model.parameters(), lr=1e-2)

# ## Set the loss function
# #loss_fn = nn.MSELoss()
# loss_fn = RMSELoss()
# # # 2 targets
# # def mse_metric(output, target):
# #     metric_epoch = mean_squared_error(output[:,:,0], target[:,:,0])
# #     return metric_epoch

# # 1 target
# def mse_metric(output, target):
#     metric_epoch = mean_squared_error(output, target, squared=False)
#     return metric_epoch

# ## Set the metric function - here using the same loss function 
# metric_fn = mse_metric #nn.MSELoss()

# ## Set the learning rate scheduler
# lr_scheduler = ReduceLROnPlateau(opt, mode='min',factor=0.5, patience=5)

# ## Set the training parameters
# params_train={
#     "num_epochs": 150,
#     "optimizer": opt,
#     "loss_func": loss_fn,
#     "metric_func": metric_fn,
#     "train_dl": train_loader, 
#     "val_dl": val_loader,
#     "sanity_check": False,
#     "lr_scheduler": lr_scheduler,
#     "path2weights": f"{path2models}/weights.pt"

# }

# ## The used device for training
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

In [89]:
# # one target
# model, loss_history = train_val(model, params_train)

In [90]:
# class CustomLSTMModel(nn.Module):
#     def __init__(self, model_params):
        
#         dyn_size  = model_params["dyn_size"]
#         hidden_size = model_params["hidden_size"]
#         output_size = model_params["output_size"]
#         number_static_predictors = model_params["number_static_predictors"]
        
#         super(CustomLSTMModel, self).__init__()
#         self.fc1 = nn.Linear(dyn_size + number_static_predictors, 256)
        
#         self.lstm = nn.LSTM(256, hidden_size, batch_first=True)
        
#           # Concatenating with static parameters
        
#         self.fc2 = nn.Linear(hidden_size, output_size)

#     def forward(self, x, static_params):
#         x1 = static_params.unsqueeze(1).expand(-1, x.shape[1], -1)
#         #print(x.shape)
#         #print(x1.shape)
#         x = torch.cat((x, x1), -1)
        
#         out = self.fc1(x)
        
#         lstm_output, _ = self.lstm(out)
        
#         # Concatenate LSTM output with static parameters
#         #combined_output = torch.cat((lstm_output, static_params.unsqueeze(1).repeat(1, lstm_output.size(1), 1)), dim=-1)
        
#         out = torch.relu(self.fc2(lstm_output))
        
        
#         return out

In [91]:
# # Training and validation 
# path2models= "./eurac_checkpoints" #./output/kaggle/working/AI4EO/models
# if not os.path.exists(path2models):
#     os.mkdir(path2models)
    
    
# ## Where to save the trained models weights 
# ## Set the optimization algorithms and learning rate
# opt = optim.Adam(model.parameters(), lr=1e-2)

# ## Set the loss function
# loss_fn = nn.MSELoss()

# ## Set the metric function - here using the same loss function 
# metric_fn = mse_metric #nn.MSELoss()

# ## Set the learning rate scheduler
# lr_scheduler = ReduceLROnPlateau(opt, mode='min',factor=0.5, patience=5)

# ## Set the training parameters
# params_train={
#     "num_epochs": 50,
#     "optimizer": opt,
#     "loss_func": loss_fn,
#     "metric_func": metric_fn,
#     "train_dl": train_loader, 
#     "val_dl": val_loader,
#     "sanity_check": False,
#     "lr_scheduler": lr_scheduler,
#     "path2weights": f"{path2models}/weights.pt"

# }

# ## The used device for training
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model_params={
#     "dyn_size": 3, #number of dynamic predictors - user_input 
#     "number_static_predictors": 85, #number of static parameters - user_input 
#     "hidden_size": 256, # user_input
#     "output_size": 2, # number_target - user_input

# }

# model = CustomLSTMModel(model_params)
# model = model.to(device)
# model

In [95]:
model_params={
    "input_size": 3, #number of dynamic predictors - user_input
    "hidden_size": 256, # user_input
    "output_size": 2, # number_target - user_input
    "number_static_predictors": 13, #number of static parameters - user_input 

}


## The used device for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = CustomLSTMModel(model_params)
model = model.to(device)
model

CustomLSTMModel(
  (lstm): LSTM(3, 256, batch_first=True)
  (fc1): Linear(in_features=269, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)

In [96]:
# Training and validation 
path2models= "./eurac_checkpoints" #./output/kaggle/working/AI4EO/models
if not os.path.exists(path2models):
    os.mkdir(path2models)
    
    
## Where to save the trained models weights 
## Set the optimization algorithms and learning rate
opt = optim.Adam(model.parameters(), lr=1e-2)

## Set the loss function
loss_fn = nn.MSELoss()

## Set the metric function - here using the same loss function 
metric_fn = mse_metric #nn.MSELoss()

## Set the learning rate scheduler
lr_scheduler = ReduceLROnPlateau(opt, mode='min',factor=0.5, patience=5)

## Set the training parameters
params_train={
    "num_epochs": 50,
    "optimizer": opt,
    "loss_func": loss_fn,
    "metric_func": metric_fn,
    "train_dl": train_loader, 
    "val_dl": val_loader,
    "sanity_check": False,
    "lr_scheduler": lr_scheduler,
    "path2weights": f"{path2models}/weights.pt"

}

In [97]:
model, loss_history = train_val(model, params_train)

Epoch 0/49, current lr=0.01
Copied best model weights!
train loss: 113.67676684505625, train metric: 14.507192611694336
val loss: 78.29340853937188, val metric: 12.184865951538086
----------
Epoch 1/49, current lr=0.01
Copied best model weights!
train loss: 80.32493791699112, train metric: 12.35503101348877
val loss: 74.94792163642497, val metric: 11.861297607421875
----------
Epoch 2/49, current lr=0.01
Copied best model weights!
train loss: 78.117589836406, train metric: 12.142365455627441
val loss: 71.03361775101762, val metric: 11.542475700378418
----------
Epoch 3/49, current lr=0.01
train loss: 76.84526354487697, train metric: 12.043232917785645
val loss: 93.298211791277, val metric: 13.408138275146484
----------
Epoch 4/49, current lr=0.01
train loss: 75.46763483699124, train metric: 11.954251289367676
val loss: 106.93161516039784, val metric: 14.428851127624512
----------
Epoch 5/49, current lr=0.01
Copied best model weights!
train loss: 70.84691213289103, train metric: 11.5648

In [58]:
model, loss_history = train_val(model, params_train)

Epoch 0/49, current lr=0.01
Copied best model weights!
train loss: 106.35827088391692, train metric: 14.203215599060059
val loss: 83.0602885940312, val metric: 12.543384552001953
----------
Epoch 1/49, current lr=0.01
Copied best model weights!
train loss: 81.36989630321017, train metric: 12.430781364440918
val loss: 78.12552666028103, val metric: 12.117409706115723
----------
Epoch 2/49, current lr=0.01
Copied best model weights!
train loss: 84.84777623269326, train metric: 12.67819595336914
val loss: 75.39499320023317, val metric: 11.87547492980957
----------
Epoch 3/49, current lr=0.01
train loss: 90.1690798695248, train metric: 13.12238883972168
val loss: 78.60841151455993, val metric: 12.202325820922852
----------
Epoch 4/49, current lr=0.01
train loss: 82.53429274808737, train metric: 12.516764640808105
val loss: 75.96803249548391, val metric: 11.920330047607422
----------
Epoch 5/49, current lr=0.01
train loss: 82.15164404795354, train metric: 12.460271835327148
val loss: 80.176

In [101]:
import numpy as np
data = np.sin(0.1 * np.arange(200)) + np.random.randn(200) * 0.1


In [102]:
data.shape

(200,)

In [103]:
seq_length = 10
num_epochs = 100
learning_rate = 0.01

# Convert data to PyTorch tensors
data = torch.FloatTensor(data).view(-1, 1)

# Create sequences for input and target
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length:i+seq_length+1]
        sequences.append((seq, target))
    return sequences

sequences = create_sequences(data, seq_length)